<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:
#!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [4]:
num_classes = 2
batch_size = 100
image_size = (224, 224)

In [5]:
generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# <---- my name in the file path is masked
training_generator = generator.flow_from_directory(
    "/xxxx/scratch/xxxx/DATA/test_data/IBM_coursera/concrete_data_week4/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)
validation_generator = generator.flow_from_directory(
    "/xxxx/scratch/xxxx/DATA/test_data/IBM_coursera/concrete_data_week4/valid",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [6]:
model_vgg16 = Sequential()
model_vgg16.add(VGG16(include_top=False, pooling='max', weights='imagenet',))
model_vgg16.add(Dense(num_classes, activation='softmax'))
model_vgg16.layers[0].trainable = False
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# import tensorflow as tf
# tf.config.list_physical_devices('GPU')

In [16]:
num_epochs = 1
L_train = len(training_generator)
L_valid = len(validation_generator)

history_vgg16 = model_vgg16.fit(training_generator, steps_per_epoch=L_train, epochs=num_epochs, 
                                validation_data=validation_generator, validation_steps=L_valid, verbose=1)

301/301 [==============================] - 2171s 7s/step - loss: 0.0555 - accuracy: 0.9897 - val_loss: 0.0534 - val_accuracy: 0.9899


In [17]:
model_vgg16.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [18]:
from tensorflow.keras.models import load_model

In [19]:
model_resnet50 = load_model('classifier_resnet_model.h5')#

In [20]:
testing_generator = generator.flow_from_directory(
    "/xxxx/scratch/xxxx/DATA/test_data/IBM_coursera/concrete_data_week4/test", target_size=image_size, shuffle=False)

Found 500 images belonging to 2 classes.


In [21]:
test_ = model_vgg16.evaluate_generator(testing_generator)
print('VGG16-trained classifier')
print('Loss: {}, Accu: {}'.format(test_[0], test_[1]))

VGG16-trained classifier
Loss: 0.05387801304459572, Accu: 0.9919999837875366


In [26]:
test_ = model_resnet50.evaluate_generator(testing_generator)
print('ResNet50-trained classifier')
print('Loss: {}, Accu: {}'.format(test_[0], test_[1]))

ResNet50-trained classifier
Loss: 0.05333923291414976, Accu: 0.9969599837064742


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [28]:
def prob_to_cate(pred):
    if pred[0] > pred[1]:
        print('Negative')
    else:
        print('Positive')

In [33]:
pred_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)

print('VGG16-trained classifier:')
for i in range(5):
    prob_to_cate(pred_vgg16[i])

VGG16-trained classifier:
Negative
Negative
Negative
Negative
Negative


In [34]:
pred_resnet = model_resnet50.predict_generator(testing_generator, steps=1)

print('VGG16-trained classifier:')
for i in range(5):
    prob_to_cate(pred_resnet[i])


VGG16-trained classifier:
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).